In [ ]:
import sys
import argparse
import torch
import cv2
import time
import re
import numpy as np
from ultralytics import YOLO
import math
import os
import pandas as pd

model_path = '/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/model_weights/pose_gauge.pt'
img_folder = "/path/to/your/images"  # Replace with the path to your image folder

def compute_angle(A, B, C):
    # ... (your existing compute_angle function)

def calculate_pointer_angle(img, coords):
    # ... (your existing calculate_pointer_angle function)

def process_images_in_folder(folder_path, output_file):
    # Get a list of all files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Create an empty DataFrame to store results
    df = pd.DataFrame(columns=['Image_Name', 'Angle_ABC', 'Angle_BDC'])

    # Process each image in the folder
    for image_file in image_files:
        # Construct the full path of the image
        image_path = os.path.join(folder_path, image_file)

        print(f"\nProcessing image: {image_path}")

        # Read the image
        frame = cv2.imread(image_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform angle calculation
        results = detectx(frame, model=model)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = plot_boxes(results, frame, classes=classes)

        # Calculate angles
        coords = results[1][0]
        angle_ABC, angle_BDC = calculate_pointer_angle(frame, coords)

        # Append results to the DataFrame
        df = df.append({'Image_Name': image_file, 'Angle_ABC': angle_ABC, 'Angle_BDC': angle_BDC}, ignore_index=True)

        # Save or display the processed image
        output_path = f"./output/result_{image_file}"
        cv2.imwrite(output_path, frame)
        print(f"Processed image saved at: {output_path}")

    # Save DataFrame to Excel
    df.to_excel(output_file, index=False)
    print(f"Results saved to: {output_file}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Process images in a folder')
    parser.add_argument('--folder', type=str, help='The path of the folder containing images')
    args = parser.parse_args()

    # Load the YOLO model
    model = torch.hub.load('/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/yolov5-master/yolov5/yolov5', 'custom', source='local', path='demo_20feb_v1.pt', force_reload=True)

    # Get class names
    classes = model.names

    # Output file for storing results
    output_file = "./output/results.xlsx"

    # Process images in the specified folder
    if args.folder:
        process_images_in_folder(args.folder, output_file)
    else:
        print("Please provide the path to the folder using the '--folder' argument.")
